In [ ]:
# Onur Can 
# Project is done for Prof. Mehmet Gönen's DASC 521: Introduction to Machine Learning @ Koç University MSc Data Science Program
# Thanks Prof Mehmet for the dataset generation and instructions

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as math

In [ ]:
#getting data into memory
data_set = np.genfromtxt("../input/sample-clothing-images-for-classification/hw02_images.csv", delimiter = ",").astype(int)
labels = np.genfromtxt("../input/sample-clothing-images-for-classification/hw02_labels.csv").astype(int)

In [ ]:
#Initilization of the data
training_images = data_set[0:30000,:]   #traning data set
test_images = data_set[30000:35000,:]   #test data set
K = np.max(labels)                      #number of classes
N = training_images.shape[0]            #data sample count
d = training_images.shape[1]            #number of features

print(data_set)
print(labels.shape,labels.size)
print(data_set.shape,training_images.shape,test_images.shape)
print(K,N,d)

In [ ]:
#Parameter estimation for training data set
#Sample mean & standart deviation calculations for xi 1-784 and for classes 1-5

sample_means = np.zeros((K,d), dtype=float)
sample_stdevs = np.zeros((K,d), dtype=float)
class_priors = np.zeros(K, dtype=float)

for i in range (K):
    current_class_set = training_images[labels[0:30000] == (i+1)]
    print("Shape of data class = ", i+1,"is equal to ",current_class_set.shape)
    class_priors[i] = current_class_set.shape[0] / N
    for j in range (d):
        feature_mean = np.mean(current_class_set[:,j])  #for class i feature j
        feature_stdev = np.std(current_class_set[:,j])  #for class i feature j
        sample_means[i][j] = feature_mean
        sample_stdevs[i][j] = feature_stdev

print("\nSample_means are listed below with the shape of the matrix\n")        
print(sample_means)
print(sample_means.shape)
print("\nSample_standart deviations are listed below with the shape of the matrix\n")        
print(sample_stdevs)
print(sample_stdevs.shape)
print("\nClass priors are listed below\n")
print(class_priors)

In [ ]:
# evaluate score functions
#for continuous naive bayes - gaussian - via Sum of  individual likelihoods

def sum_x_i_likelihoods(X,mean,stdeviation,dimension,Klass):      #sum of individual Likelihoods
    xi_individual_likelihoods = np.zeros(dimension)
    for i in range (0,dimension):
        L = (-0.5) * np.log(2 * math.pi * stdeviation[Klass][i]**2) - (0.5) * (
            X[0][i] - mean[Klass][i])**2 / stdeviation[Klass][i]**2
        xi_individual_likelihoods[i] = L
    sum_of_L = np.sum(xi_individual_likelihoods)
    return sum_of_L

#Score values for classes
def class_score(K,training_set,mean_set,deviation_set,d,priors):  # gx = sum(log(likelihoods) + log(prior) for each x
    score_array = np.zeros(5)
    for j in range(K):
        score_array[j] = sum_x_i_likelihoods(training_set,mean_set,deviation_set,d,j) + np.log(priors[j])
    return score_array

score_matrix = np.zeros((N,5))
for k in range(N):
    score_matrix[k] = (class_score(K,training_images[k,None],sample_means,sample_stdevs,d,class_priors))
print(score_matrix,score_matrix.shape)
    
    

#print(class_score(K,training_images[0,None],sample_means,sample_stdevs,d,class_priors))
#print(sum_x_i_likelihoods(training_images[0,None],sample_means[0,None],sample_stdevs[0,None],d,0))

In [ ]:
#Score Values for the Test Values
N_test = test_images.shape[0]            #test data sample count
test_score_matrix = np.zeros((N_test,5))
for k in range(N_test):
    test_score_matrix[k] = (class_score(K,test_images[k,None],sample_means,sample_stdevs,d,class_priors))
print(test_score_matrix,test_score_matrix.shape)

In [ ]:
#CONFUSION MATRIX for training data set
y_predicted_training = np.zeros(N).astype(int)

traning_labels = labels[0:30000]


for h in range(N):
    b = np.max(score_matrix[h])
    c = (score_matrix[h] == b)
    for g in range(5):
          if c[g] == True:
            y_predicted_training[h] = g + 1
            break
print(y_predicted_training.shape)

confusion_matrix = pd.crosstab(y_predicted_training, traning_labels, rownames = ['y_pred'], colnames = ['y_truth'])    #PANDAS
print(confusion_matrix)


In [ ]:
#CONFUSION MATRIX for test data set
y_predicted_test = np.zeros(N_test).astype(int)

test_labels = labels[30000:35000]


for h in range(N_test):
    b = np.max(test_score_matrix[h])
    c = (test_score_matrix[h] == b)
    for g in range(5):
          if c[g] == True:
            y_predicted_test[h] = g + 1
            break
print(y_predicted_test.shape)

confusion_matrix = pd.crosstab(y_predicted_test, test_labels, rownames = ['y_pred'], colnames = ['y_truth'])    #PANDAS
print(confusion_matrix)
